In [1]:
import os
os.environ['Qdrant_API_KEY'] = 'KaqAcYuLeNIvUxWMQJisJrqcm2rERYB381zqbuWWz_0pFxB2rkZ18A'
os.environ['GROQ_API_KEY'] = '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'

In [2]:
import pandas as pd
import numpy as np

In [3]:
import pandas as pd

df = pd.read_csv("/Users/kundankumar/Downloads/Bali_Hotel_Review.csv", delimiter=';')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5798 entries, 0 to 5797
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       5798 non-null   object 
 1   Hotel          5798 non-null   object 
 2   UserID         5798 non-null   object 
 3   Title          5798 non-null   object 
 4   Review         5798 non-null   object 
 5   Rating         5798 non-null   int64  
 6   Value          5787 non-null   float64
 7   Accessibility  5786 non-null   float64
 8   Service        5787 non-null   float64
 9   Room           5787 non-null   float64
 10  Cleanliness    5787 non-null   float64
 11  Sleep Quality  5783 non-null   float64
dtypes: float64(6), int64(1), object(5)
memory usage: 543.7+ KB


In [5]:
import pandas as pd

# Function to format each row into a single string
def format_row(row):
    return (
        f"Location: {row['Location']}, Hotel: {row['Hotel']}, UserID: {row['UserID']}, "
        f"Title: {row['Title']}, Review: {row['Review']}, Rating: {row['Rating']}, "
        f"Value: {row['Value']}, Accessibility: {row['Accessibility']}, Service: {row['Service']}, "
        f"Room: {row['Room']}, Cleanliness: {row['Cleanliness']}, Sleep Quality: {row['Sleep Quality']}"
        "\n\n".lower()
    )

# Apply the function to each row and create a new column with the formatted text
df['formatted_text'] = df.apply(format_row, axis=1)

# Convert the formatted text into a list (or any other format you need)
text_data = df['formatted_text'].tolist()


In [6]:
text_data

['location: legian kuta bali, hotel: merccure bali legian, userid: barry sloan, title: nice hotek, review: check in was smooth although my room wasn’t ready at 5pm, i was due to arrive at 4pm first night i was in a room overlooking jl legian which is a very busy load st, i asked if i could be moved the 2nd night and they were very helpful and moved to a great room overlooking 1 of the pools. overall the staff here are very friendly and helpful, rooms are comfortable. breakfast buffet is good with plenty to choose from. water pressure is bad so it’s a little frustrating getting a good shower. great location but be aware there is a lot of traffic noise. i’d still happily stay here again, rating: 4, value: 0.0, accessibility: 0.0, service: 1.0, room: 0.0, cleanliness: 0.0, sleep quality: 0.0\n\n',
 'location: legian kuta bali, hotel: merccure bali legian, userid: u4ianoosa, title: perfect location, great facilities, review: we thoroughly enjoyed our stay at mercure legian, with a fantasti

In [7]:
from random import shuffle
sampled_dataset = text_data
shuffle(text_data)
sampled_dataset = text_data[:800]

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-large-en-v1.5")


/opt/anaconda3/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
vectors = model.encode(sampled_dataset)

In [11]:
vectors[0].shape

(1024,)

In [12]:
from qdrant_client import QdrantClient
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(
    url="https://a39527c9-207f-4db4-82b9-6d8039f3c72d.us-east4-0.gcp.cloud.qdrant.io:6333", 
    api_key="KaqAcYuLeNIvUxWMQJisJrqcm2rERYB381zqbuWWz_0pFxB2rkZ18A",
)

In [13]:
client.recreate_collection(
    collection_name="try_data",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

/var/folders/jh/4_kmjddj2737hhv79vl44jsr0000gn/T/ipykernel_12303/430421355.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [15]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

client.upload_collection(
    collection_name="try_data",
    ids=[i for i in range(len(sampled_dataset))],
    vectors=vectors,
    parallel=4,    
    max_retries=3,
)


In [16]:
from dspy.retrieve.qdrant_rm import QdrantRM
qdrant_retriever_model = QdrantRM("try_data", client, k=3)

In [17]:
import os
import dspy


# Initialize the GROQ object with the API key
llama3 = dspy.GROQ(model='llama3-8b-8192', api_key=os.environ['GROQ_API_KEY'])


In [18]:
dspy.settings.configure(rm=qdrant_retriever_model, lm=llama3)

In [19]:
# Define the GenerateAnswer signature for the model
class GenerateAnswer(dspy.Signature):
    """Answer questions with logical factoid answers."""
    context = dspy.InputField(desc="will contain try hotels data of bali matched with the query")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="an answer between 20 to 30 words")
    reasons = dspy.OutputField(desc="reasons for including each hotel in the answer")


In [20]:
# Define a function to retrieve context
def get_context(text):
    query_vector = model.encode(text)
    hits = client.search(
        collection_name="try_data",
        query_vector=query_vector,
        limit=3
    )
    s = ''
    for x in [sampled_dataset[i.id] for i in hits]:
        s = s + x
    return s


In [21]:
# Define RAG module
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

# Initialize RAG module
rag = RAG()

In [22]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()


        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)


    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [23]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
   
    def respond(query, chat_history):
        response = rag(query)
        chat_history.append((query, response.answer))
        return "", chat_history


    msg.submit(respond, [msg, chatbot], [msg, chatbot])

In [24]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://3e95502ba56e150d6d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
